In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/gianluca/Documents/GitHub/tesi/violence-detection/notebooks/dataset.csv')
data.groupby("class")["num_frames"].describe()

,count,mean,std,min,25%,50%,75%,max
class,,,,,,,,
NonViolence,1579.0,101.818873,234.948969,0.0,26.0,34.0,37.0,2620.0
Violence,766.0,71.317232,69.682322,0.0,33.0,38.0,100.0,400.0


In [13]:
import os
from tqdm import tqdm

def getelements(path):
    items = os.listdir(path)

    if '.DS_Store' in items:
        items.remove('.DS_Store')

    return items

def copy_video(source: str, dest: str, frames:list, pattern: str):
    for i, frame in enumerate(frames, 1):
        ext = frame.split('.')[-1]
        filename = pattern % i
        filename += '.' + ext
        os.renames(os.path.join(source, frame), os.path.join(dest, filename))


def make_dataset(source_path: str, dest_path: str, min_frames: int, max_frames: int, pattern: str = 'frame_%06d'):
    cls = getelements(source_path)

    for cl in cls:
        print(f'Class: {cl}')
        dirs = getelements(os.path.join(source_path, cl))
        video_name = 1
        for dir in tqdm(dirs):
            frames = sorted(getelements(os.path.join(source_path, cl, dir)))
            num_frames = len(frames)

            if num_frames > max_frames:
                for i in range(0, len(frames), max_frames):
                    chunk = frames[i:i + max_frames]
                    if len(chunk) >= min_frames:
                        copy_video(os.path.join(source_path, cl, str(dir)), os.path.join(dest_path, cl, str(video_name)), chunk, pattern)
                        video_name += 1
            elif num_frames >= min_frames:
                copy_video(os.path.join(source_path, cl, str(dir)), os.path.join(dest_path, cl, str(video_name)), frames, pattern)
                video_name += 1

In [14]:
make_dataset("/Users/gianluca/Documents/GitHub/tesi/bus_dataset_bin", "./dataset", 16, 48, 'frame_%02d')

Class: NonViolence


100%|██████████| 1579/1579 [00:23<00:00, 67.14it/s] 


Class: Violence


100%|██████████| 766/766 [00:06<00:00, 116.43it/s]


In [2]:
data = pd.read_csv('/Users/gianluca/Documents/GitHub/tesi/violence-detection/notebooks/dataset_mod.csv')
data.groupby("class")["num_frames"].describe()

,count,mean,std,min,25%,50%,75%,max
class,,,,,,,,
NonViolence,3878.0,41.178700,9.767495,16.0,34.0,48.0,48.0,48.0
Violence,1306.0,40.534456,10.064427,16.0,34.0,48.0,48.0,48.0


In [3]:
data.groupby(['class', 'num_frames']).size()

class        num_frames
NonViolence  16             34
             17             31
             18             56
             19             55
             20             54
                          ... 
Violence     44              6
             45              8
             46              2
             47              1
             48            755
Length: 66, dtype: int64

In [9]:
import numpy as np

minCount = data.groupby(['class']).size().min()
minCount

nonViolence = os.listdir('./dataset/NonViolence')
if '.DS_Store' in nonViolence:
    nonViolence.remove('.DS_Store')

nonViolence = list(np.random.choice(nonViolence, size=minCount, replace=False))
nonViolence.sort(key=int)

In [11]:
from tqdm import tqdm

path = './dataset/NonViolence'
dest = './dataset/NonViolence_new'
videos = os.listdir(path)
if '.DS_Store' in videos:
    videos.remove('.DS_Store')
videos.sort(key=int)

for video in tqdm(videos):
    if video in nonViolence:
        os.renames(os.path.join(path, video), os.path.join(dest, video))

100%|██████████| 3878/3878 [00:00<00:00, 9213.72it/s]


In [14]:
data = pd.read_csv('/Users/gianluca/Documents/GitHub/tesi/violence-detection/notebooks/dataset_balanced.csv')
data.groupby("class")["num_frames"].describe()

,count,mean,std,min,25%,50%,75%,max
class,,,,,,,,
NonViolence,1306.0,41.042879,9.822938,16.0,34.0,48.0,48.0,48.0
Violence,1306.0,40.534456,10.064427,16.0,34.0,48.0,48.0,48.0


In [1]:
import cv2
from tqdm import tqdm

frame_data = './dataset'

cls = os.listdir(frame_data)
if '.DS_Store' in cls:
        cls.remove('.DS_Store')

for cl in cls:
    base = os.path.join(frame_data, cl)
    videos = os.listdir(base)
    if '.DS_Store' in videos:
        videos.remove('.DS_Store')
    print(f'Class: {cl}')

    for video in tqdm(videos):
        video_base = os.path.join(base, video)
        frames = sorted(os.listdir(video_base))
        if '.DS_Store' in frames:
            frames.remove('.DS_Store')

        frame = cv2.imread(os.path.join(video_base, frames[0]))
        height, width, layers = frame.shape

        video_name = os.path.join(base, cl.upper() + '_' + video + '.avi')
        out = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'MPEG'), 30, (width, height))
        for frame in frames:
            img = cv2.imread(os.path.join(video_base, frame))
            out.write(img)
        out.release()       

Class: NonViolence


100%|██████████| 1306/1306 [30:12<00:00,  1.39s/it]


Class: Violence


100%|██████████| 1306/1306 [32:00<00:00,  1.47s/it]
